環境変数`GOOGLE_APPLICATION_CREDENTIALS`が設定されている必要がある。
値は👇

In [3]:
!set | grep GOOGLE

BASH_EXECUTION_STRING='set | grep GOOGLE'
GOOGLE_APPLICATION_CREDENTIALS=/Users/hotoku/.config/gcloud/legacy_credentials/dtws_yasunori_horikoshi@oaklawn.co.jp/adc.json


In [1]:
from google.cloud import bigquery

client = bigquery.Client(
    project="olm-pipeline-dev",
    location="asia-northeast1"
)

In [17]:
jobs = client.list_jobs(all_users=True)
itr = iter(jobs)
job = next(itr)

In [22]:
[l for l in dir(job) if not l[0] == "_"]

['add_done_callback',
 'allow_large_results',
 'billing_tier',
 'cache_hit',
 'cancel',
 'cancelled',
 'clustering_fields',
 'create_disposition',
 'created',
 'ddl_operation_performed',
 'ddl_target_routine',
 'ddl_target_table',
 'default_dataset',
 'destination',
 'destination_encryption_configuration',
 'done',
 'dry_run',
 'ended',
 'error_result',
 'errors',
 'estimated_bytes_processed',
 'etag',
 'exception',
 'exists',
 'flatten_results',
 'from_api_repr',
 'job_id',
 'job_type',
 'labels',
 'location',
 'maximum_billing_tier',
 'maximum_bytes_billed',
 'num_child_jobs',
 'num_dml_affected_rows',
 'parent_job_id',
 'path',
 'priority',
 'project',
 'query',
 'query_parameters',
 'query_plan',
 'range_partitioning',
 'referenced_tables',
 'reload',
 'result',
 'running',
 'schema_update_options',
 'script_statistics',
 'self_link',
 'set_exception',
 'set_result',
 'slot_millis',
 'started',
 'state',
 'statement_type',
 'table_definitions',
 'time_partitioning',
 'timeline',
 '

In [34]:
print(f"{job.total_bytes_billed=}, {job.cache_hit=}, {job.job_type=}, {job.started=}, {job.job_id=}")

job.total_bytes_billed=20971520, job.cache_hit=False, job.job_type='query', job.started=datetime.datetime(2020, 11, 20, 3, 33, 9, 136000, tzinfo=<UTC>), job.job_id='job_cQBXX9TGV8p_w-Zf89oUSSrG4oFx'


In [45]:
from datetime import datetime, timezone
import pandas as pd

In [49]:
jobs = iter(client.list_jobs(all_users=True))
_df = {}
for job in jobs:
    if job.job_type != "query":
        continue
    if job.cache_hit:
        continue
    if (datetime.now(tz=timezone.utc) - job.started).days > 10:
        break
    _df[job.job_id] = dict(
        total_bytes_billed = job.total_bytes_billed,
        started=job.started,
        job_id=job.job_id,
        query=job.query
    )
df = pd.DataFrame.from_dict(_df, orient="index")    

In [51]:
len(df)

34098

In [55]:
df["scan_gib"] = df.total_bytes_billed / (1024**3)
df

,total_bytes_billed,started,job_id,query,scan,scan_gib
job_x7Rj1JN_mVkrbXsTIwY43Ob3WHaU,3.145728e+07,2020-11-20 03:53:14.495000+00:00,job_x7Rj1JN_mVkrbXsTIwY43Ob3WHaU,"\n\nwith temp1 as (\nselect\n l.uuid,\n l.la...",0.031457,0.029297
job_OQ4viGD4zhARb6sYfJ0eH3lW59Dq,3.145728e+07,2020-11-20 03:53:14.469000+00:00,job_OQ4viGD4zhARb6sYfJ0eH3lW59Dq,"\n\nwith temp1 as (\nselect\n l.uuid,\n l.la...",0.031457,0.029297
job_NFRYDITud3b-OzhurC3hmOk5CNnm,3.145728e+07,2020-11-20 03:53:14.166000+00:00,job_NFRYDITud3b-OzhurC3hmOk5CNnm,"\n\nwith temp1 as (\nselect\n l.uuid,\n l.la...",0.031457,0.029297
job_gCX3yRTJHgXKo-KVII4vHUJvc_mg,2.097152e+07,2020-11-20 03:52:59.577000+00:00,job_gCX3yRTJHgXKo-KVII4vHUJvc_mg,"\n\ncreate temp function mymin(x float64, y fl...",0.020972,0.019531
job_EI5wFoGVBTVLwKJmyCHeKLDPTyDt,2.097152e+07,2020-11-20 03:52:59.495000+00:00,job_EI5wFoGVBTVLwKJmyCHeKLDPTyDt,"\n\ncreate temp function mymin(x float64, y fl...",0.020972,0.019531
...,...,...,...,...,...,...
5de899ba-7800-4c8f-9423-5869956e5ec0,1.048576e+07,2020-11-09 13:02:42.884000+00:00,5de899ba-7800-4c8f-9423-5869956e5ec0,#standardSQL\n\nCREATE TEMP FUNCTION geoToH3(l...,0.010486,0.009766
job_mVkV-OwEOsfrmt9O2rahvygTdWOC,1.121662e+10,2020-11-09 13:00:17.711000+00:00,job_mVkV-OwEOsfrmt9O2rahvygTdWOC,#standardSQL\n\nCREATE TEMP FUNCTION geoToH3(l...,11.216617,10.446289
bqjob_r46a6039d15a1dbde_00000175ab870305_1,0.000000e+00,2020-11-09 05:42:34.001000+00:00,bqjob_r46a6039d15a1dbde_00000175ab870305_1,"#standardSQL\n\nwith t as (\n (select 1 as x,...",0.000000,0.000000
bqjob_r1320679255804d15_00000175ab86e616_1,NaN,2020-11-09 05:42:26.470000+00:00,bqjob_r1320679255804d15_00000175ab86e616_1,"#standardSQL\n\nwith t as (\n (select 1 as x,...",NaN,NaN
